In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import torch
from mpl_toolkits import mplot3d

import crossmodal
import fannypack

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
# Choose experiment to eval
# experiment_name = "pf_blackout0.0_new"
experiment_name = "pf_blackout0.0_nommpretrain"

In [55]:
# Create Buddy, model
buddy = fannypack.utils.Buddy(experiment_name)
model_type = buddy.metadata["model_type"]
dataset_args = buddy.metadata["dataset_args"]
filter_model = crossmodal.door_models.model_types[model_type]()
buddy.attach_model(filter_model)

[buddy-pf_blackout0.0_nommpretrain] Using device: cuda
[buddy-pf_blackout0.0_nommpretrain] Loaded metadata: {'commit_hash': 'eef3b3cb92090d1025d8dcd7980ab52d92635a72', 'dataset_args': {'image_blackout_ratio': 0.0, 'sequential_image_rate': 1, 'use_haptics': True, 'use_proprioception': True, 'use_vision': True}, 'eval_results': {'raw_rmse': [0.020570801571011543, 0.018060145899653435, 0.012222472578287125], 'theta_rmse_deg': 0.46530788158420383, 'x_rmse_cm': 0.1020448631137479, 'y_rmse_cm': 0.06906894809044897}, 'model_type': 'DoorParticleFilter', 'notes': 'Commented out pretraining for measurement model', 'train_end_time': 'Jun 16, 2020 @ 3:53:53', 'train_start_time': 'Jun 16, 2020 @ 2:26:00'}
[buddy-pf_blackout0.0_nommpretrain] Available checkpoint labels: ['0000000000090698', '0000000000091890', '0000000000093111', '0000000000094283', '0000000000095572', 'phase0', 'phase1', 'phase2', 'phase3']


In [56]:
# Load a trajectory
fannypack.data.set_cache_path(crossmodal.__path__[0] + "/../.cache")
trajectories = crossmodal.door_data.load_trajectories(
    ("panda_door_pull_10.hdf5", 1), **dataset_args
)

[TrajectoriesFile-...door_pull_10.hdf5] Loading trajectory from file: <HDF5 file "1sO3avSEtegDcgISHdALDMW59b-knRRvf-panda_door_pull_10.hdf5" (mode r)>
[TrajectoriesFile-...door_pull_10.hdf5] Existing trajectory count: 10
[TrajectoriesFile-...door_pull_10.hdf5] Opening file...
[TrajectoriesFile-...door_pull_10.hdf5] Closing file...


In [57]:
# Pull out trajectory
states, observations, controls = trajectories[0]

print("States shape:", states.shape)
print("Controls shape:", controls.shape)

States shape: (800, 3)
Controls shape: (800, 7)


In [58]:
span = 2.0
resolution = 0.02

# Time step to visualize at
t = 35

# Create delta
grid_delta = np.mgrid[
    -span / 2.0 : span / 2.0 : resolution, -span / 2.0 : span / 2.0 : resolution
]
_, grid_cols, grid_rows = grid_delta.shape
grid_delta = grid_delta.T.reshape((1, grid_cols * grid_rows, 2))
zero_col = np.zeros((1, grid_cols * grid_rows, 1))
grid_delta = np.concatenate((zero_col, grid_delta), axis=2)
assert grid_delta.shape == (1, grid_cols * grid_rows, 3)

# Get state, observation
state_input = fannypack.utils.to_torch(
    states[t : t + 1, None] + grid_delta, device=buddy.device
)
observation_input = fannypack.utils.to_torch(
    fannypack.utils.SliceWrapper(observations)[t : t + 1], device=buddy.device
)

# Enable grad computation for inputs
for v in observation_input.values():
    v.requires_grad = True

In [59]:
print(observation_input.keys())
observation_input["image"].shape
plt.figure()
plt.imshow(fannypack.utils.to_numpy(observation_input["image"][0]), cmap="gray")
plt.show()

dict_keys(['gripper_pos', 'gripper_sensors', 'image'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
# Load latest checkpoint
buddy.load_checkpoint()
log_weights = filter_model.measurement_model(
    states=state_input, observations=observation_input
)

# # Load checkpoint after pretraining
# buddy.load_checkpoint("phase2")
# log_weights_pretrain = filter_model.measurement_model(
#     states=state_input, observations=observation_input
# )

[buddy-pf_blackout0.0_nommpretrain] Read checkpoint from path: checkpoints/pf_blackout0.0_nommpretrain-phase3.ckpt
[buddy-pf_blackout0.0_nommpretrain] Instantiating optimizer:  train_dynamics_single_step
[buddy-pf_blackout0.0_nommpretrain] Instantiating optimizer:  train_dynamics_recurrent
[buddy-pf_blackout0.0_nommpretrain] Instantiating optimizer:  train_filter_recurrent
[buddy-pf_blackout0.0_nommpretrain] Loaded checkpoint at step: 96215


In [63]:
def process_weights(log_weights):
    weights = np.exp(fannypack.utils.to_numpy(log_weights))
    weights = weights.reshape((grid_cols, grid_rows)) / np.max(weights)
    return weights


weights = process_weights(log_weights)
# weights_pretrain = process_weights(log_weights_pretrain)

In [64]:
fig = plt.figure()
ax = plt.axes(projection="3d")

x = grid_delta[0, :, 1].reshape((grid_cols, grid_rows))
y = grid_delta[0, :, 2].reshape((grid_cols, grid_rows))

ax.plot_surface(x, y, weights, cmap="viridis", edgecolor="none", alpha=1.0)
# ax.plot_surface(x, y, weights_pretrain, cmap="plasma", edgecolor="none", alpha=0.3)

ax.set_title("Door Hinge X/Y Likelihood Map")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
fig = plt.figure()
ax = plt.axes(projection="3d")

x = grid_delta[0, :, 1].reshape((grid_cols, grid_rows))
y = grid_delta[0, :, 2].reshape((grid_cols, grid_rows))

ax.plot_surface(x, y, weights, cmap="viridis", edgecolor="none", alpha=1.0)
# ax.plot_surface(x, y, weights_pretrain, cmap="plasma", edgecolor="none", alpha=0.3)

ax.set_title("Door Hinge X/Y Likelihood Map")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
# Compute saliency
torch.mean(log_weights).backward(retain_graph=True)

In [52]:
image, saliency_map = fannypack.utils.to_numpy([
    observation_input["image"][0, :, :],
    observation_input["image"].grad[0, :, :]
])
saliency_map = np.abs(saliency_map)

fig, axes = plt.subplots(1, 2)
axes = axes.flatten()
axes[0].imshow(image)
axes[1].imshow(saliency_map)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
print(observation_input["gripper_pos"].grad)
print(observation_input["gripper_sensors"].grad)

tensor([[ 0.1658,  0.0734, -0.1259]], device='cuda:0')
tensor([[ 0.1305,  0.4751, -0.1491,  0.0474, -0.0628,  0.4591, -0.6401]],
       device='cuda:0')
